In [6]:
import pandas as pd
from glob import glob

In [22]:
def read_act_and_sim(isotope, 
                     act_path = '/home/usc/ie/mpm/NEXT100-0nubb-analysis/nexus_job_templates/ft3/backgrounds/activities_efficiencies{}.ods',
                     st_path = '/mnt/lustre/scratch/nlsas//home/usc/ie/mpm/NEXT100/data/radiogenics/{}/prod/volumes/event_stats.txt',
                     vol_path = '/mnt/lustre/scratch/nlsas//home/usc/ie/mpm/NEXT100/data/radiogenics/{}/prod/volumes/{}/', 
                     act_version = ''):
    st_path = st_path.format(isotope)

    year = (3600 * 24 * 365) / 1000

    activities = dict(pd.read_excel(act_path.format(act_version), sheet_name=isotope, engine='odf').set_index("G4Volume").TotalActivity)

    with open(st_path, 'r') as f:
        text = f.readlines()
        f.close()
    sim_evs     = {t.split(' ')[-4].split('/')[-1]:int(t.split(' ')[-2])*int(t.split(' ')[1]) for t in text}
    sim_evs_red = {t.split(' ')[-4].split('/')[-1]:int(t.split(' ')[-2])*len(glob(vol_path.format(isotope, t.split(' ')[-4].split('/')[-1]) + 'prod/hypathia/*')) for t in text}
    activities = {k: activities[k] for k in sorted(activities)}
    sim_evs = {k: sim_evs[k] for k in sorted(sim_evs)}
    sim_evs_red = {k: sim_evs_red[k] for k in sorted(sim_evs_red)}

    act = pd.DataFrame([activities]).T.reset_index().rename(columns = {'index': 'vol', 0:'activities'})
    sim = pd.DataFrame([sim_evs]).T.reset_index().rename(columns = {'index': 'vol', 0:'sim'})
    sim_red = pd.DataFrame([sim_evs_red]).T.reset_index().rename(columns = {'index': 'vol', 0:'sim_red'})
    sim_act = act.merge(sim, on = 'vol')
    sim_act = sim_act.merge(sim_red, on = 'vol')
    sim_act['exposure'] = sim_act.sim / (sim_act.activities * year)
    sim_act['reduced_exp'] = sim_act.sim_red / (sim_act.activities * year)
    
    return sim_act

In [34]:
def make_exp_df(isotope):
    old = read_act_and_sim(isotope, act_version = '').drop(['sim_red', 'reduced_exp'], axis = 1)[['vol', 'sim', 'activities', 'exposure']].rename(columns={'activities':'old_act', 'exposure':'old_exp'})
    new = read_act_and_sim(isotope, act_version = '_upd2').drop(['sim_red', 'reduced_exp'], axis = 1)[['vol', 'sim', 'activities', 'exposure']].rename(columns={'activities':'new_act', 'exposure':'new_exp'})
    exp_df = old.merge(new, on = ['vol', 'sim'])
    exp_df['isotope'] = isotope
    return exp_df

In [45]:
make_exp_df('214Bi').append(make_exp_df('208Tl')).to_hdf('/mnt/lustre/scratch/nlsas//home/usc/ie/mpm/NEXT100/data/radiogenics/exposures.h5', 'exposures')

In [46]:
pd.read_hdf('/mnt/lustre/scratch/nlsas//home/usc/ie/mpm/NEXT100/data/radiogenics/exposures.h5', 'exposures')

,vol,sim,old_act,old_exp,new_act,new_exp,isotope
0,ANODE_RING,6127152,4.760,40.817377,1.176242,165.179246,214Bi
1,CATHODE_RING,5872545,4.610,40.394182,1.141094,163.191733,214Bi
2,DB_PLUG,199920050,155.000,40.899505,155.120000,40.867865,214Bi
3,EP_COPPER_PLATE,386100000,306.000,40.010296,82.021879,149.266889,214Bi
4,FIELD_RING,40409712,32.000,40.043236,27.962356,45.825308,214Bi
5,GATE_RING,6096272,4.760,40.611663,1.176124,164.363228,214Bi
6,HDPE_TUBE,3938760,3.070,40.683147,3.070860,40.671753,214Bi
7,ICS,236432040,187.000,40.092036,53.015926,141.414314,214Bi
8,LIGHT_TUBE,2517076,1.990,40.108521,3.545623,22.511125,214Bi
9,OPTICAL_PAD,10165771,7.920,40.701324,7.920000,40.701324,214Bi


In [37]:
from invisible_cities.database.load_db import RadioactivityData

In [43]:
RadioactivityData('next100')[0]

,G4Volume,Isotope,TotalActivity
0,ANODE_RING,208Tl,0.360
1,ANODE_RING,214Bi,4.760
2,CATHODE_RING,208Tl,0.349
3,CATHODE_RING,214Bi,4.610
4,DB_PLUG,208Tl,118.000
5,DB_PLUG,214Bi,155.000
6,EP_COPPER_PLATE,208Tl,91.000
7,EP_COPPER_PLATE,214Bi,306.000
8,FIELD_RING,208Tl,24.900
9,FIELD_RING,214Bi,32.000
